In [31]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import ast

# Cargar el modelo y el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")


In [32]:
df_train = pd.read_csv("../data/NER_SA_csvs/train.csv")
df_val = pd.read_csv("../data/NER_SA_csvs/validation.csv")
df_test = pd.read_csv("../data/NER_SA_csvs/test.csv")


In [33]:
sentences_train = df_train['sentence'].tolist()
sentences_val = df_val['sentence'].tolist()
sentences_test = df_test['sentence'].tolist()


In [34]:
# Lista de oraciones (de tu dataset)

# Función para obtener embeddings para todas las oraciones
def get_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        # Tokenizar la frase
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
        
        # Desactivar el cálculo de gradientes (porque no estamos entrenando)
        with torch.no_grad():
            # Obtener las representaciones de la última capa (embeddings)
            outputs = model(**inputs)
            word_embeddings = outputs.last_hidden_state  # (batch_size, sequence_length, hidden_size)
        
        # Extraer los embeddings de la última capa (generalmente la capa [CLS] es el "representante" de la oración, pero si necesitas todos los tokens, puedes usar todos los embeddings)
        embeddings.append(word_embeddings)
    return embeddings


In [35]:
# Obtener los embeddings para todas las oraciones
sentence_embeddings_train = get_embeddings(sentences_train)
sentence_embeddings_val = get_embeddings(sentences_val)
sentence_embeddings_test = get_embeddings(sentences_test)

In [36]:
df_train['embeddings'] = sentence_embeddings_train
df_val['embeddings'] = sentence_embeddings_val
df_test['embeddings'] = sentence_embeddings_test


In [37]:
df_train.to_csv("../data/NER_SA_csvs/train.csv", index=False)
df_val.to_csv("../data/NER_SA_csvs/validation.csv", index=False)
df_test.to_csv("../data/NER_SA_csvs/test.csv", index=False)